In [1]:
from boson_data_lib import *
import numpy as np
import scipy.linalg as sl
import os
import pandas as pd
import glob

In [2]:
data_dir = "../DATA/"
tests_dir = "../TESTS/"

In [3]:
best_init_rho =  np.array([[ 0.96693975+0.j        , -0.17832301+0.00560344j],
                               [-0.17832301-0.00560344j,  0.03306025+0.j        ]])

fid_dists = fidelity_distances(data_dir, best_init_rho);

In [12]:
def read_as_dataframe(filename, fid_distances, data_dir, tests_dir):

    df = pd.DataFrame()
    
    print("Processing...")
    
    with h5py.File(tests_dir+filename, "r") as f:
        
        gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33']
        #gammas = f.keys()
        
        for gamma in gammas:
            
            print(gamma)
            
            #g = gamma[6:]
            
            init_states = ["State_D"+str(i) for i in range(1, 21)] #f[gamma].keys()
            
            for state in init_states:
                
                Fidelity = f['gamma_'+gamma][state]["Fidelity"][()]

                ser_len = len(Fidelity)

                gamma_column = [gamma] * ser_len
                state_column = [state[7:]] * ser_len

                fid_dist_column = [fid_distances[int(state[7:])-1]] * ser_len      

                f_name = data_dir + "/" + state + "_2CUT_data.h5"
                #f_name = data_dir + "\\" + state + "_2CUT_data.h5"
                t, dt = extract_time(f_name, gamma)
                time_column = t
                gamma_time_column = np.array(t)*float(gamma) 

                d_ser = {'Gamma': gamma_column,
                         'State': state_column,
                         'Time': time_column, 
                         'gt': gamma_time_column,
                         'Fidelity': Fidelity,
                         'Infidelity': 1-Fidelity,
                         'Distance': fid_dist_column}

                df_ser = pd.DataFrame(data = d_ser)   
                df = pd.concat([df, df_ser])
    
    print(" done!")
    
    return df

In [13]:
#test_file = "POP_LME_6_random_trn4_tst20_2024-Jan-16_at_11-11.h5" #complex Hamiltonian
#test_file = "POP_LME_6_random_trn4_tst20_2024-Jan-16_at_11-29.h5" #simple Hamiltonian
test_file = "POP_LME_1_full_trn4_tst20_2024-Jan-17_at_11-29.h5"

In [14]:
df = read_as_dataframe("../TESTS/"+test_file, fid_dists, data_dir, tests_dir)

Processing...
0.079477
0.25133
0.79477
2.5133
7.9477
25.133
79.477
251.33
 done!


In [15]:
df

,Gamma,State,Time,gt,Fidelity,Infidelity,Distance
0,0.079477,1,0.06000,0.004769,1.000000,-6.039613e-14,0.341925
1,0.079477,1,0.08000,0.006358,1.000000,1.013490e-08,0.341925
2,0.079477,1,0.10000,0.007948,1.000000,1.509112e-07,0.341925
3,0.079477,1,0.12000,0.009537,1.000000,8.047366e-08,0.341925
4,0.079477,1,0.14000,0.011127,0.999996,3.742306e-06,0.341925
...,...,...,...,...,...,...,...
383,251.33,20,0.14600,36.694180,0.998500,1.499752e-03,0.340013
384,251.33,20,0.14625,36.757013,0.998502,1.497979e-03,0.340013
385,251.33,20,0.14650,36.819845,0.998504,1.495964e-03,0.340013
386,251.33,20,0.14675,36.882677,0.998506,1.493668e-03,0.340013


In [16]:
pkl_name = "dataframe_"+test_file[:-3]+".pkl"

df.to_pickle(tests_dir+pkl_name)

In [17]:
pkl_name

'dataframe_POP_LME_1_full_trn4_tst20_2024-Jan-17_at_11-29.pkl'

In [18]:
tests_dir+pkl_name

'../TESTS/dataframe_POP_LME_1_full_trn4_tst20_2024-Jan-17_at_11-29.pkl'